# Using Muldoon

Here are some examples for how to use some of the functionality of Muldoon.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from numpy.random import normal

from scipy.stats import median_abs_deviation as mad

from muldoon.met_timeseries import MetTimeseries, PressureTimeseries, TemperatureTimeseries
from muldoon.utils import modified_lorentzian, fit_vortex_ps, write_out_plot_data
from muldoon.read_data import *

Processing label: https://pds-atmospheres.nmsu.edu/PDS/data/PDS4/Mars2020/mars2020_meda/data_derived_env/sol_0000_0089/sol_0001/WE__0001___________DER_PS__________________P02.xml
Found a Header structure: HEADER
Found a Table_Delimited structure: TABLE
0   Header           'HEADER'  PDS DSV 1
1   Table_Delimited  'TABLE'   6 fields x 2330 records
[16.09203194 16.0923025  16.09257278 ... 16.96741806 16.96768861
 16.96795889]


In [2]:
# Test that the sub-second sampling scheme works

filename="WE__0089___________CAL_ATS_________________P01.CSV"
time = make_seconds_since_midnight(filename, subsecond_sampling=True)
plt.plot(time)
delta_time = time[1:] - time[:-1]

# No overlapping times, right?
assert(len(delta_time[delta_time == 0]) == 0)

TypeError: make_seconds_since_midnight() got an unexpected keyword argument 'subsecond_sampling'

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax2 = ax.twinx()

# How to read in data

# Temperature data
filename="WE__0089___________CAL_ATS_________________P01.CSV"
temperature_time, temperature = read_Perseverance_ATS_data(filename, which_ATS=1)
ax.scatter(temperature_time, temperature)

# Pressure data
filename="WE__0089___________DER_PS__________________P01.CSV"
pressure_time, pressure = read_Perseverance_PS_data(filename)
ax2.scatter(pressure_time, pressure, color='orange')

In [ ]:
# Create time-series
time = np.linspace(-10, 10, 1000)
baseline = 0.
slope = 1.
t0 = 0.
DeltaP = 1.
Gamma = 0.5

right_answer = np.array([baseline, slope, t0, DeltaP, Gamma])

profile = modified_lorentzian(time, baseline, slope, t0, DeltaP, Gamma) + normal(scale=slope/20., size=len(time))
mt = PressureTimeseries(time, profile)

In [ ]:
# Detrend
window_size = Gamma
detrended_pressure = mt.detrend_timeseries_boxcar(window_size)
print(np.std(mt.detrended_data))
print(np.isclose(np.std(mt.detrended_data), 0.1, atol=0.1))

plt.scatter(mt.time, mt.data, marker='.')
plt.scatter(mt.time, mt.detrended_data, marker='.')

In [ ]:
# Test time-series write-out
write_str = write_out_plot_data(mt.time, mt.data, yerr=np.std(mt.data)*np.ones_like(mt.data),
                                x_label="Time", y_label="Pressure", test_mode=True)

In [ ]:
# plt.plot(mt.time, profile)
conv = mt.apply_lorentzian_matched_filter(2.*mt.sampling, 1./np.pi)
mx_ind = np.argmax(mt.convolution)

print(mt.convolution[mx_ind], np.abs(mt.time[mx_ind]), 2.*mt.sampling)
# plt.plot(mt.time, mt.convolution)
# plt.plot(mt.time, profile)
plt.plot(mt.convolution)
# Make sure convolution returns a strong peak at the right time
print((np.abs(mt.time[mx_ind])))

In [ ]:
# Test find_vortices - Set the threshold very low to make sure the right peak is actually at the top of the list
vortices = mt.find_vortices(detection_threshold=2)
plt.plot(mt.convolution)
plt.axvline(mt.peak_indices[0] - 3.*mt.peak_widths[0])
plt.axvline(mt.peak_indices[0] + 3.*mt.peak_widths[0])

# Make sure the max peak in the convolution is the right one
assert(mt.time[mt.peak_indices[0]] < Gamma)

In [ ]:
# Test find_vortices
vortices = mt.find_vortices(detection_threshold=7)
plt.plot(mt.time, mt.data)
plt.axvline(mt.time[mt.peak_indices[0] - 3*int(mt.peak_widths[0])])
plt.axvline(mt.time[mt.peak_indices[0] + 3*int(mt.peak_widths[0])])

In [ ]:
plt.scatter(vortices[0]["time"], vortices[0]["data"])

# Test vortex fit
old_popt, old_unc = fit_vortex(vortices[0], [0., 1., 0., 1., 0.01], 
                          [[-1, -1, np.min(vortices[0]["time"]), 0, 0],
                           [1, 1, np.max(vortices[0]["time"]), 2, 1]], sigma=vortices[0]["scatter"])
assert(np.max(np.abs(old_popt - right_answer)/old_unc) < 3.)

plt.plot(vortices[0]["time"], modified_lorentzian(vortices[0]["time"], *old_popt))

In [ ]:
init_params = mt._determine_init_params(vortices[0])
bounds = mt._determine_bounds(vortices[0], init_params)
popt, unc = fit_vortex(vortices[0], init_params, bounds, sigma=vortices[0]["scatter"])
assert(np.max(np.abs(popt - right_answer)/unc) < 3.)

print(np.max(np.abs(popt - right_answer)/unc))

In [ ]:
# Test fit all vortices

# Create time-series
time = np.linspace(-10, 10, 1000)
baseline = 0.
slope = 1.
t0 = 0.
DeltaP = 1.
Gamma = 0.5

right_answer = np.array([baseline, slope, t0, DeltaP, Gamma])

profile = modified_lorentzian(time, baseline, slope, t0, DeltaP, Gamma) + normal(scale=slope/20., size=len(time))
mt = PressureTimeseries(time, profile)
window_size = Gamma
detrended_pressure = mt.detrend_timeseries_boxcar(window_size)

conv = mt.apply_lorentzian_matched_filter(2.*mt.sampling, 1./np.pi)
vortices = mt.find_vortices(detection_threshold=7)

popts, uncs = mt.fit_all_vortices()
print(np.max(np.abs(popts[0] - right_answer)/uncs[0]))
assert(np.max(np.abs(popts[0] - right_answer)/uncs[0]) < 5.)

In [ ]:
# Make plot showing how the data are conditioned and write out the data in each panel
mt.make_conditioned_data_figure(write_filename="./test_");

In [ ]:
# Create time-series
time = np.linspace(-10, 10, 1000)
baseline = 0.
slope = 1.
t0 = 0.
DeltaP = 1.
Gamma = 0.5

DeltaT = -10.

right_answer = np.array([baseline, slope, t0, DeltaP, Gamma])
temp_right_answer = np.array([baseline, slope, t0, DeltaT, Gamma])

profile = modified_lorentzian(time, baseline, slope, t0, DeltaP, Gamma) + normal(scale=slope/20., size=len(time))
mt = PressureTimeseries(time, profile)

# Add in some nasty sine wave action
red_noise_period = Gamma
temp_profile = modified_lorentzian(time, *temp_right_answer) +\
    normal(scale=slope, size=len(time)) + DeltaT/10.*np.sin(2*np.pi/red_noise_period*time)
tt = TemperatureTimeseries(time, temp_profile, 
                           popts=[right_answer], 
                           uncs=[np.array([0, 0, 0, 0, 0])])
plt.scatter(tt.time, tt.data)

plt.axvline(tt.popts[0][2])

In [ ]:
tt.retrieve_vortices()
plt.scatter(tt.vortices[0]["time"], tt.vortices[0]["data"])
print(len(tt.vortices[0]["time"]))

In [ ]:
# Fit vortex using normal linear regression
# baseline, slope, t0, DeltaT, Gamma

temp_baseline = np.mean(tt.vortices[0]["data"])
temp_avg_slope = (tt.vortices[0]["data"][-1] - tt.vortices[0]["data"][0])/\
    (tt.vortices[0]["time"][-1] - tt.vortices[0]["time"][0])
max_excursion = -np.abs(np.max(tt.vortices[0]["data"]) - np.min(tt.vortices[0]["data"]))

sigma = mad(tt.vortices[0]["data"])

init_params = [temp_baseline, temp_avg_slope, t0, max_excursion, Gamma]
bounds = [[np.min(tt.vortices[0]["data"]), -3.*np.abs(temp_avg_slope), np.min(tt.vortices[0]["time"]), 
           -3.*np.abs(max_excursion), 0.], 
          [np.max(tt.vortices[0]["data"]),  3.*np.abs(temp_avg_slope), np.max(tt.vortices[0]["time"]), 
            0., 3.*Gamma]]

popt, unc = fit_vortex(tt.vortices[0], init_params, bounds)
print(popt)
print(unc)
print(temp_right_answer)

plt.scatter(tt.vortices[0]["time"], tt.vortices[0]["data"])
plt.plot(tt.vortices[0]["time"], modified_lorentzian(tt.vortices[0]["time"], *popt), lw=3, color='orange')

In [ ]:
# Fit vortex using Gaussian processes - https://george.readthedocs.io/en/latest/tutorials/model/

---
Using the TemperatureTimeseries Class

In [ ]:
# Temperature data
filename="WE__0089___________CAL_ATS_________________P01.CSV"

In [ ]:
time, temperature = read_Perseverance_ATS_data(filename, which_ATS=1)
tt = TemperatureTimeseries(time, temperature)

plt.scatter(tt.time, tt.data, marker='.')

In [ ]:
# Detrend
window_size = 1. # 1 hour
detrended_temperature = tt.detrend_timeseries_boxcar(window_size)
plt.scatter(tt.time, tt.detrended_data, marker='.')